# Know thy Higgs boson

**Background:** High-energy collisions at the Large Hadron Collider (LHC) produce particles that interact with particle detectors. One important task is to classify different types of collisions based on their physics content, allowing physicists to find patterns in the data and to potentially unravel new discoveries.

**Problem statement:** The discovery of the Higgs boson by CMS and ATLAS Collaborations was announced at CERN in 2012. In this work, we focus on the potential of Machine Learning and Deep Learning in detecting potential Higgs signal from one of the background processes that mimics it.

**Dataset:** The dataset is made available by the Center for Machine Learning and Intelligent Systems at University of California, Irvine. The dataset can be found on the [UCI Machine learning Repository](https://archive.ics.uci.edu/ml/datasets/HIGGS)

**Description:** The dataset consists of a total of 11 million labeled samples of Higgs vs background events produced by Monte Carlo simulations. Each sample consists of 28 features. The first 21 features are kinematic properties measured at the level of the detectors. The last seven are functions of the first 21.

**Steps to load the training dataset**
1. **Download the dataset from the UCI website**

In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz

--2025-06-15 11:01:20--  https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: 'HIGGS.csv.gz'

HIGGS.csv.gz            [                <=> ]   2.62G  22.8MB/s    in 2m 20s  

2025-06-15 11:03:41 (19.2 MB/s) - 'HIGGS.csv.gz' saved [2816407858]



The above will create a file named `HIGGS.csv.gz`. Remember that in particle physics we deal with large amounts of data. In this case as well you have to be patient till the full file of `2.62 GB` data is downloaded. For a full analysis you might need to use the full dataset but for this exercise you can try with `10000` entries only

2. **Unzip the folder**

In [2]:
 !gzip -d -f HIGGS.csv.gz 

The above will create a `HIGGS.csv` file in your local directory. At this point you will see that the `HIGGS.csv.gz` file is unzipped as `HIGGS.csv` and got deleted after unzipping is complete.

3. **Load libraries**

If something is missing install them in the python environment

In [3]:
from sklearn.datasets import make_gaussian_quantiles
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    ConfusionMatrixDisplay 
)

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

import numpy as np
np.random.seed(42)  # for reproducibility
import h5py

from sklearn.metrics import roc_curve, auc

from itertools import combinations
import matplotlib.pyplot as plt
import seaborn as sns

from xgboost import XGBClassifier

import warnings


/cvmfs/cms.cern.ch/el9_amd64_gcc12/external/py3-numpy/1.24.3-a3a65cbc18c6a98e06451d4329e3ea47/lib/python3.9/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/cvmfs/cms.cern.ch/el9_amd64_gcc12/external/py3-numpy/1.24.3-a3a65cbc18c6a98e06451d4329e3ea47/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/cvmfs/cms.cern.ch/el9_amd64_gcc12/external/py3-numpy/1.24.3-a3a65cbc18c6a98e06451d4329e3ea47/lib/python3.9/site-packages/numpy/core/getlimits.py:518: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/cvmfs/cms.cern.ch/el9_amd64_gcc12/external/py3-numpy/1.24.3-a3a65cbc18c6a98e06451d4329e3ea47/lib/python3.9/site-pac

4. **Load the file using pandas library**

In [ ]:
data=pd.read_csv('./HIGGS.csv', header=None)
## print the first 5 rows
data.head()

The first column is the class label (1 for signal, 0 for background), followed by the 28 features (21 low-level features then 7 high-level features): lepton pT, lepton eta, lepton phi, missing energy magnitude, missing energy phi, jet 1 pt, jet 1 eta, jet 1 phi, jet 1 b-tag, jet 2 pt, jet 2 eta, jet 2 phi, jet 2 b-tag, jet 3 pt, jet 3 eta, jet 3 phi, jet 3 b-tag, jet 4 pt, jet 4 eta, jet 4 phi, jet 4 b-tag, m_jj, m_jjj, m_lv, m_jlv, m_bb, m_wbb, m_wwbb. **Try to find the meaning of each of the variables from the datasource**

In [ ]:
## change the column numbers to column names
data.columns = ['target',
              'lep_pT',
              'lep_eta',
              'lep_phi',
              'missE',
              'missphi',
              'jet1_pt',
              'jet1_eta',
              'jet_1_phi',
              'jet_1_b_tag',
              'jet_2_pt',
              'jet_2_eta',
              'jet_2_phi',
              'jet_2_b_tag',
              'jet_3_pt',
              'jet_3_eta',
              'jet_3_phi',
              'jet_3_b_tag',
              'jet_4_pt',
              'jet_4_eta',
              'jet_4_phi',
              'jet_4_b_tag',
              'm_jj',
              'm_jjj',
              'm_lv',
              'm_jlv',
              'm_bb',
              'm_wbb',
              'm_wwbb']

## print the first 5 rows (including the column names)
data.head()

Assign first column $0$ to class labels (labeled $1$ for signal, $0$ for background)  and all others to feature matrix $X$.

In this example, for the sake of fast checking, we use `10000` samples. To train on the entire dataset, proceed with uncommenting the lines below.

In [ ]:
X=data.iloc[:10000,1:]#data.iloc[:,1:]
y=data.iloc[:10000,0]#data.iloc[:,0]

X_train1, X_val, y_train1, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X_train1, y_train1, test_size=0.2, random_state=42)

5. **Visualize your data - One histogram per feature column**

Detailed information on what each feature column is can be found in *Attribute Information* section on the [UCI Machine learning Repositery](https://archive.ics.uci.edu/ml/datasets/HIGGS). For further information, refer to the [paper](https://www.nature.com/articles/ncomms5308) by Baldi et. al

In [ ]:
fig, axes = plt.subplots(len(X_train.columns)//3, 3, figsize=(12, 48))

i = 0
for triaxis in axes:
    for axis in triaxis:
        X_train.hist(column = X_train.columns[i], bins = 100, ax=axis)
        i = i+1

You can also visualise it through `pairplot`, but only select a few features at a time otherwise it would take a long time to produce the pairplot. In the following I have presented an example of pairplot. Use other features to see the pairplot.

In [ ]:
# Select only the eta and phi columns for leptons and jets, plus the target for classification
selected_columns = ['target',
              'lep_pT',
              #'lep_eta',
              #'lep_phi',
              #'missE',
              #'missphi',
              'jet1_pt',
              #'jet1_eta',
              #'jet_1_phi',
              #'jet_1_b_tag',
              'jet_2_pt',
              #'jet_2_eta',
              #'jet_2_phi',
              #'jet_2_b_tag',
              'jet_3_pt',
              #'jet_3_eta',
              #'jet_3_phi',
              #'jet_3_b_tag',
              'jet_4_pt',
              #'jet_4_eta',
              #'jet_4_phi',
              #'jet_4_b_tag',
              #'m_jj',
              #'m_jjj',
              #'m_lv',
              #'m_jlv',
              #'m_bb',
              #'m_wbb',
              #'m_wwbb'
              ]

# Select only those subset of the data
pairplot_data = data[:500][selected_columns] # only taking the first 5000 data points


plt.figure(figsize=(8, 8), dpi=250)
sns.pairplot(
    pairplot_data,
    hue="target",
    palette="Set1",
    diag_kind="hist",
    # Make dots smaller
    plot_kws={"s": 10},
)
plt.suptitle("Pairplot of features", y=1.02)
plt.show()

## **Setup the Boosted Decision Tree model** 

Boosted Decision Trees are explaned here [here](https://xgboost.readthedocs.io/en/stable/tutorials/model.html). This are what can be classified as `ensemble learning methods`, The main idea is to combine many weak learners (typically shallow decision trees) to create a strong classifier. In the following we will use **XGBoost = "Extreme Gradient Boosting"** package. 

For each round (up to `n_estimators`):
1. **Compute Gradient (error/loss derivative):** Measures how wrong the current prediction is using the weak learners with `max_depth`. This sets the maximum depth of each individual decision tree (not the number of trees).

- A tree of depth $3$ has at most $2^3 = 8$ leaf nodes.
- It controls the complexity of each tree:
    - Smaller depth → simpler trees, less overfitting.
    - Larger depth → more complex trees, more capacity, but risk of overfitting.

**Intuition:** If each tree is a `correction rule`, `max_depth` limits how detailed each rule can be.

2. **Fit a New Tree:** A decision tree is trained to predict the gradient (residual) of the loss function.

3. **Update Predictions:** Add the tree’s output to the model’s prediction (with some scaling factor: learning_rate).

4. **Regularization:** Penalize complex trees to avoid overfitting (e.g., with max_depth, min_child_weight, gamma).




In [ ]:
classifier = XGBClassifier(
    n_estimators=200,
    max_depth=3,
    learning_rate=0.1,
    use_label_encoder=False,
    eval_metric='logloss'  # or 'mlogloss' for multi-class
)

In [ ]:
warnings.filterwarnings("ignore") # I am suppressing some warning, but you can let it run
classifier.fit(X_train, y_train)

Now it is the time to predict the class using the classifier and see how it performs. 

**Objective: Improve the model accuracy**

In [ ]:
# Make predictions
predictions = classifier.predict(X_test)

# Confusion Matrix, but this time using seaborn
cm = confusion_matrix(y_test, predictions)

# Plot the confusion matrix
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.tight_layout()
plt.show()

# Compute and print scores
acc = accuracy_score(y_test, predictions)
prec = precision_score(y_test, predictions, average='binary')  
rec = recall_score(y_test, predictions, average='binary')
f1 = f1_score(y_test, predictions, average='binary')

print(f"Accuracy  : {acc:.4f}")
print(f"Precision : {prec:.4f}")
print(f"Recall    : {rec:.4f}")
print(f"F1 Score  : {f1:.4f}")

## Setup the neural network model

**Objective:** Use pytorch to build a network to predict the classes and compare that with bossted tree output.

**Hints:**
- you might want to use the Binary cross entropy loss.

In [ ]:
import torch
import torch.nn as nn

class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(29, 100),
            nn.ReLU(),
            nn.Linear(100, 300),
            nn.ReLU(),
            nn.Linear(300, 100),
            nn.ReLU(),
            nn.Linear(100, 29),
            nn.ReLU(),
            nn.Linear(29, 1)
        )
    def forward(self, x):
        return self.net(x)

model = SimpleNet()
criterion = nn.BCELoss()
optimizer = torch.optim.BGD(model.parameters(), lr=learning_rate)

In [ ]:
!pip3 install torch